In [1]:
import pandas as pd
import os
import librosa
import numpy as np
import scipy
import re
import math

import collections

#Pickling
from six.moves import cPickle as pickle

# Normalizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
#HTTP
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.style as ms
import librosa.display
import IPython.display as ipd
import seaborn
ms.use('seaborn-muted')
%matplotlib inline

x_train = []
y_train = []
x_test = []
y_test = []

no_alignment_file = [4764,3977]
wrong_alignment = [3730]

In [6]:
def extract_patterns(data,extract=False):
    if(extract):
        patterns = {}
        for index, row in data.iterrows():
            text = row['text']
            if('[' in row['text']):
                print('Found in ',text)
                text = ' '.join([x for x in text.split(' ') if '[' not in x])
                print('Now ',text)
            # MAybe remove the laughter actions etc
            patterns[row['index']] = set(get_pattern([text])[0].values())
            print('Extracted pattern from '+ row['index'] + ' index:'+ str(index))
            print('Size: ', len(patterns[row['index']]), 'Patterns size', len(patterns))
        try:
            print('Saving Pickle')
            with open('pickles/patterns/pattern.pickle','wb') as f:
                save = {
                    'patterns' : patterns
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
                print('Successfully saved in pattern.pickle')
                return patterns
        except Exception as e:
            print('Unable to save data to pickle', e)
            print('Patterns probably not saved.')
            return patterns
    else:
        try:
            with open('pickles/patterns/pattern.pickle','rb') as f:
                save = pickle.load(f)
                patterns = save['patterns']
                del save
                returning = {}
                for key in list(data['index']):
                    returning[key] = patterns[key]
                return returning
        except Exception as e:
            print('Error loading base datasets pickle: ', e)
            

def get_pattern(text):
    text = json.dumps(text)
    url = 'http://192.168.2.101:7878/api/get_patt'
    data = dict(input_tweets = text)
    resp = requests.post(url=url, data=data)
    r = json.loads(resp.text)
    return map(lambda x: x['pattern'],r)
    
def get_word_segments(align_file):
    segments = []
    try:
        with open(align_file,'r') as openFile:
            read_lines = openFile.readlines()
            lines = read_lines[1:len(read_lines)-1]
            for line in lines:
                splitted = line.split()
                segments.append({'sf': splitted[0], 'ef':splitted[1],'seg_scr':splitted[2],'word':splitted[3]})
    except Exception as e:
        print('Problem with file ',align_file)
    return segments

def clean_text(text):
    punct_str = '!"#$%&()*+,-./:;<=>?@\\^_`{|}~«»“…‘”\t'
    for p in punct_str:
        text = text.replace(p,' ')
    text = re.sub(' +', ' ', text)
    return text.lower().strip()
    
def extract_patterns(data,extract=False):
    if(extract):
        patterns = {}
        for index, row in data.iterrows():
            patterns[row['index']] = set(get_pattern([row['text']])[0].values())
            print('Extracted pattern from '+ row['index'] + ' index:'+ str(index))
            print('Size: ', len(patterns[row['index']]), 'Patterns size', len(patterns))
        try:
            print('Saving Pickle')
            with open('pickles/patterns/pattern.pickle','wb') as f:
                save = {
                    'patterns' : patterns
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
                print('Successfully saved in pattern.pickle')
                return patterns
        except Exception as e:
            print('Unable to save data to pickle', e)
            print('Patterns probably not saved.')
            return patterns
    else:
        try:
            with open('pickles/patterns/pattern.pickle','rb') as f:
                save = pickle.load(f)
                patterns = save['patterns']
                del save
                returning = {}
                for key in list(data['index']):
                    returning[key] = patterns[key]
                return returning
        except Exception as e:
            print('Error loading base datasets pickle: ', e)
            
def filter_word_count(data, n_count):
    return data[list(map(lambda x: len(x.split(' ')) >= n_count,data['text']))]

def remove_empty_patterns(data,patterns):
    empty_patterns = [k for k, v in patterns.items() if len(v) < 1]
    patterns = { k:v for k, v in patterns.items() if len(v) >= 1 }
    data = filter(lambda x: x[1]['index'] not in empty_patterns ,data.iterrows())
    data = pd.DataFrame.from_items(data).T
    return data,patterns

def remove_multiwildcard(patterns):
    for index, patt in patterns.items():
        flt_patt = {p for p in patt if p.split(' ').count('.+') == 1}
        patterns[index] = flt_patt
    return patterns

# Feature Extractors

In [15]:
def get_mfcc(y,sr,se_tup,n_mfcc=20):
    mfcc = librosa.feature.mfcc(y=y[se_tup[0]:se_tup[1]],sr=sr, n_mfcc=n_mfcc)
    mfcc = np.mean(mfcc,axis=1)
    return mfcc


def get_rmse(y,se_tup):
    return np.sqrt(np.mean(y[se_tup[0]:se_tup[1]]**2))

def get_zcr(y,se_tup):
    zcr = librosa.feature.zero_crossing_rate(y[se_tup[0]:se_tup[1]]  + 0.0100) # 0.0200 background noise reduction
    return zcr[0] 

def extract_features(y,sr, se_tup):
#     results = [
#         get_mfcc(y,sr,se_tup,n_mfcc=1),
#         get_mfcc(y,sr,se_tup),
#         get_rmse(y,se_tup),
#         get_zcr(y,se_tup)
#     ]
    results = get_rmse(y,se_tup)
    return results
    

In [94]:
def load_data(word_count,emotional_mapping):
    # full = generate_IEMOCAP_df()
    data = pd.read_csv('data/IEMOCAP_sentences_votebased.csv',index_col=0)
    data['emotion_code'] = data['emotion'].map( emotional_mapping ).astype(int)
    # Take away fear, surprise,disgust, xxx and others. Not enough data
    data = data[data.emotion_code < 4]
    #Remove rows that don't have Alignment file
    try:
        data = data.drop(no_alignment_file)
    except Exception as e:
        print('Error at: ',e)
    # Remove rows that have wrong Alignment file
    try:
        data = data.drop(wrong_alignment)
    except Exception as e:
        print('Error at: ',e)
    # Clean Transcripts
    data['text'] = data['text'].apply(clean_text)
    # Filter Word Count
#     data = filter_word_count(data, word_count)
    patterns = extract_patterns(data)
    data,patterns = remove_empty_patterns(data,patterns)
    patterns = remove_multiwildcard(patterns)
    return data,patterns

def get_word_coordinate(word_segments,transcript, y_size):
    word_data = []
    start_data = []
    end_data = []
    silence = ['<sil>']
    actions = ['++garbage++','++breathing++', '++laughter++','++lipsmack++']
    if(len(word_segments) > 0 ):
        frames = list(map(lambda x: (x['sf'],x['ef']),word_segments))
        frame_size = int(frames[len(frames)-1][1])
        rate = y_size / frame_size
        word_segments.pop()
        word_segments.pop(0)
        text_tokens = transcript.split(' ')
        token_index = 0
        for wordsg in word_segments:
            word = wordsg['word'].lower().split('(')[0]
            if(word in silence):
                'There is a silence here'
            elif(word in actions):
                action = word
                token_index += 1
            else:
                word_data.append(word)
                start_data.append(int(math.floor(int(wordsg['sf'])* rate)))
                end_data.append(int(math.ceil(int(wordsg['ef'])* rate)))
                
                if(word != text_tokens[token_index]):
                    print(transcript)
                    print(word_segments)
                    print('ERROR')
                    print('Expected', text_tokens[token_index])
                    print('Found', word)
                    return None
                token_index +=1
        return [word_data,start_data,end_data]
    return None

# OLD VERSION
# def matchForcedAlign(patt,parsedData):
#     wild_card = '.+'
#     i = 0
#     initWildCardCount = 0
#     endWildCardCount = 0
#     wordCount = 0
#     word_index = []
#     transcript = parsedData[0]
#     passedWord = False
#     pattern_tokens = patt.split(' ')
#     for token in pattern_tokens:
#         if(token == wild_card):
#             if(passedWord):
#                 endWildCardCount += 1
#                 # TODO: Fix instance of [con_w wild_card con_w]
#             else:
#                 initWildCardCount += 1
#         else:
#             passedWord = True
#             wordCount += 1
#             if word_index == []:
#                 word_index = [i for i, e in enumerate(transcript) if e == token]
#     results = []
#     index_removal = []
#     for index in word_index:
#         if((index - initWildCardCount) >= 0 and (index + wordCount - 1 + endWildCardCount) < len(transcript)):
#             index_result = [(-1,-1),(-1,-1),(-1,-1)]
#             if(initWildCardCount > 0):
#                 index_result[0] = (parsedData[1][index-initWildCardCount],parsedData[2][index - 1])
#             if(endWildCardCount > 0):
#                 index_result[2] = (parsedData[1][index + wordCount],parsedData[2][index+ wordCount - 1 + endWildCardCount])
#             index_result[1] = (parsedData[1][index],parsedData[2][index + wordCount - 1])
#             results.append(index_result)
#         else:
#             index_removal.append(index)
#     for index in index_removal:
#         word_index.remove(index)
#     return word_index, results, [initWildCardCount,wordCount,endWildCardCount]

def matchForcedAlign(patt,parsedData):
    wild_card = '.+'
    i = 0
    pattern_types = []
    word_index = []
    transcript = parsedData[0]
    passedWord = False
    pattern_tokens = patt.split(' ')
    for token in pattern_tokens:
        if(token == wild_card):
            pattern_types.append(0)
        else:
            pattern_types.append(1)
            if(not passedWord):
                passedWord = True
                word_index = [i for i, e in enumerate(transcript) if e == token]

    results = []
    patt_len = len(pattern_tokens)
    index_removal = []
#     print('Word index',word_index)
#     print('Pattern types',pattern_types)
    for index in word_index:
#         print('Starting index',index - pattern_types.index(1))
#         print('Ending Index', index + len(pattern_types[pattern_types.index(1):]) - 1)
        
        if((index - pattern_types.index(1)) >= 0 and (index + len(pattern_types[pattern_types.index(1):]) - 1) < len(transcript)):
            start = index - pattern_types.index(1)
            index_result = []
            for i, pt in enumerate(pattern_types):

                index_result.append((parsedData[1][start],parsedData[2][start]))
                start += 1

            results.append(index_result)
        else:
            index_removal.append(index)
    for index in index_removal:
        word_index.remove(index)
    word_index = [i - pattern_types.index(1) for i in word_index]
    return word_index, results, pattern_types

In [274]:
emotional_mapping = {'ang': 0, 'sad': 1, 'hap': 2, 'neu': 3,'fru': 4,'exc': 5,'fea': 6,'sur': 7,'dis': 8, 'xxx':9,'oth':10}
data,patterns = load_data(2,emotional_mapping)
data.groupby('emotion').count()['index']

Error at:  labels [4764 3977] not contained in axis


emotion
ang    1178
hap     723
neu    1566
sad    1008
Name: index, dtype: int64

In [173]:
def get_features(patterns, extractFeatures = False):
    feature_table = None
    if(extractFeatures):
        wcards_b = False
        connectw_b = False
        all_patt = True
        feature_table = {}

        for pattern_name in patterns:
            row = data[data['index'] == pattern_name]
            emotion = row.emotion.values[0]
            wav_path = row.wav_path.values[0]
            align_path = row.alignment_path.values[0]
            segments = get_word_segments(align_path)
            transcript = row.text.values[0]
#             print(transcript)
            y,sr = librosa.load(wav_path)
            parsedData = get_word_coordinate(segments,transcript,len(y))
            for patt in patterns[pattern_name]: 
                word_index, results, pattern_types = matchForcedAlign(patt,parsedData)
                features_vect = None
                for index, result in enumerate(results):
                    if(all_patt):
                        x_temp = result[0][0]
                        y_temp = result[len(result)-1][1]
                        '''
                         x_temp and y_temp get the beginning and end of the pattern inside the transcript
                        '''    
                        all_patt_feature = extract_features(y,sr,(x_temp,y_temp))
                        features_vect = all_patt_feature
                
                    if(wcards_b):
                        wc_loc = pattern_types.index(0)
                        x_temp = result[wc_loc][0]
                        y_temp = result[wc_loc][1]
                        
                        wcard_feature = extract_features(y,sr,(x_temp,y_temp))
                        features_vect = wcard_feature
                    if(connectw_b):
                        cw_loc = [i for i, pt in enumerate(pattern_types) if pt == 1]
                        cword_features = [extract_features(y,sr,(result[i][0],result[i][1])) for i in cw_loc]
                        cword_feature = np.mean(cword_features)
                        features_vect = cword_feature
                    if(pattern_name not in feature_table):
                        feature_table[pattern_name] = {}
                    feature_table[pattern_name][(str(index)+'_'+ patt)] = features_vect
                print('Extracted feature from pattern',patt,'in recording',pattern_name)
#                 for index,features in enumerate(features_vect):
#                     feature_table[(pattern_name, patt,index)] = features

        try:
            print('Saving Pickle')
            with open('pickles/patterns/full_rmse_features4emo.pickle','wb') as f:
                save = {
                    'feature_table' : feature_table
                }
                pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
                print('Successfully saved in pattern_features4emo.pickle')
                return feature_table
        except Exception as e:
            print('Unable to save data to pickle', e)
    else:
        try:
            with open('pickles/patterns/rmse_features4emo.pickle','rb') as f:
                save = pickle.load(f)
                feature_table = save['feature_table']
                del save
                return feature_table
        except Exception as e:
            print('Error loading pattern features pickle: ', e)

# feature_table = get_features(patterns,extractFeatures= True)

In [174]:
# feature_table

In [141]:
cw_feature_table = None
wc_feature_table = None
full_feature_table = None
try:
    with open('pickles/patterns/cw_rmse_features4emo.pickle','rb') as f:
        save = pickle.load(f)
        cw_feature_table = save['feature_table']
        del save
    with open('pickles/patterns/wc_rmse_features4emo.pickle','rb') as f:
        save = pickle.load(f)
        wc_feature_table = save['feature_table']
        del save
    with open('pickles/patterns/full_rmse_features4emo.pickle','rb') as f:
        save = pickle.load(f)
        full_feature_table = save['feature_table']
        del save
except Exception as e:
    print('Error loading pattern features pickle: ', e)

# Scaler

In [223]:
# MFCC_INDEX = 0
# RMSE_INDEX = 1
# ZCR_INDEX = 2
# MFCCscaler = MinMaxScaler(feature_range=(0,1))
# RMSEscaler = MinMaxScaler(feature_range=(0,1))
# ZCRscaler = MinMaxScaler(feature_range=(0,1))
fullscaler = MinMaxScaler(feature_range=(0,1))
wcscaler = MinMaxScaler(feature_range=(0,1))
cwscaler = MinMaxScaler(feature_range=(0,1))

full_data = np.array([item for sub in full_feature_table.values() for item in list(sub.values())]).reshape(-1,1)
fullscaler.fit(full_data)

wc_data = np.array([item for sub in wc_feature_table.values() for item in list(sub.values())]).reshape(-1,1)
wcscaler.fit(wc_data)

cw_data = np.array([item for sub in cw_feature_table.values() for item in list(sub.values())]).reshape(-1,1)
cwscaler.fit(cw_data)


MinMaxScaler(copy=True, feature_range=(0, 1))

# Normalize 

In [238]:
scalfull_feature_table = {}
scalwc_feature_table = {}
scalcw_feature_table = {}
print('Starting')
for key in full_feature_table:
    feature_data = full_feature_table[key]
    scaled_feature_data = {}
    for pattern in feature_data:
        feat = fullscaler.transform(feature_data[pattern])
        scaled_feature_data[pattern] = feat[0][0]
    scalfull_feature_table[key] = scaled_feature_data
    print('Finished ',key)
print('All done')

for key in cw_feature_table:
    feature_data = cw_feature_table[key]
    scaled_feature_data = {}
    for pattern in feature_data:
        feat = cwscaler.transform(feature_data[pattern])
        scaled_feature_data[pattern] = feat[0][0]
    scalcw_feature_table[key] = scaled_feature_data
    print('Finished ', key)
print('All done')

for key in wc_feature_table:
    feature_data = wc_feature_table[key]
    scaled_feature_data = {}
    for pattern in feature_data:
        feat = wcscaler.transform(feature_data[pattern])
        scaled_feature_data[pattern] = feat[0][0]
    scalwc_feature_table[key] = scaled_feature_data
    print('Finished ', key)
print('All done')
        
        

#         scaled_feature_data[pattern] = feat_list
#     scaled_feature_table[key] = scaled_feature_data
#     print('Finished ', key)
# print('All done')

# for key in cw_feature_table:
#     scaled_feature_data = {}
#     feature_data = feature_table[key]
#     for pattern in feature_data:
#         print(pattern)
#         mfcc_feat = scalers[MFCC_INDEX].transform(feature_data[pattern][MFCC_INDEX].reshape(1,-1))
#         rmse_feat = scalers[RMSE_INDEX].transform(feature_data[pattern][RMSE_INDEX])
#         zcr_array = np.array(feature_data[pattern][ZCR_INDEX])
#         zcr_feat = np.array([np.min(zcr_array),np.mean(zcr_array), np.max(zcr_array)])
#         feat_list = np.hstack((mfcc_feat[0],rmse_feat[0],zcr_feat))
#         scaled_feature_data[pattern] = feat_list
#     scaled_feature_table[key] = scaled_feature_data
#     print('Finished ', key)
# print('All done')

# for key in wc_feature_table:
#     scaled_feature_data = {}
#     feature_data = feature_table[key]
#     for pattern in feature_data:
#         print(pattern)
#         mfcc_feat = scalers[MFCC_INDEX].transform(feature_data[pattern][MFCC_INDEX].reshape(1,-1))
#         rmse_feat = scalers[RMSE_INDEX].transform(feature_data[pattern][RMSE_INDEX])
#         zcr_array = np.array(feature_data[pattern][ZCR_INDEX])
#         zcr_feat = np.array([np.min(zcr_array),np.mean(zcr_array), np.max(zcr_array)])
#         feat_list = np.hstack((mfcc_feat[0],rmse_feat[0],zcr_feat))
#         scaled_feature_data[pattern] = feat_list
#     scaled_feature_table[key] = scaled_feature_data
#     print('Finished ', key)
# print('All done')
# full_feature_table['Ses01F_impro01_F002'].values()
# fullscaler.transform()

Starting
Finished  Ses01F_impro01_F000
Finished  Ses01F_impro01_F002
Finished  Ses01F_impro01_F005
Finished  Ses01F_impro01_F012
Finished  Ses01F_impro01_M011
Finished  Ses01F_impro01_M013
Finished  Ses01F_impro02_F000
Finished  Ses01F_impro02_F001
Finished  Ses01F_impro02_F002
Finished  Ses01F_impro02_F004
Finished  Ses01F_impro02_F005
Finished  Ses01F_impro02_F007
Finished  Ses01F_impro02_F010
Finished  Ses01F_impro02_F011
Finished  Ses01F_impro02_F012
Finished  Ses01F_impro02_F013
Finished  Ses01F_impro02_F014
Finished  Ses01F_impro02_F015
Finished  Ses01F_impro02_F017
Finished  Ses01F_impro02_F019
Finished  Ses01F_impro02_F020
Finished  Ses01F_impro02_M000
Finished  Ses01F_impro02_M001
Finished  Ses01F_impro02_M002
Finished  Ses01F_impro02_M004
Finished  Ses01F_impro02_M005
Finished  Ses01F_impro02_M009
Finished  Ses01F_impro02_M010
Finished  Ses01F_impro02_M013
Finished  Ses01F_impro02_M014
Finished  Ses01F_impro03_F000
Finished  Ses01F_impro03_F001
Finished  Ses01F_impro03_F002
F

Finished  Ses01F_script03_2_F037
Finished  Ses01F_script03_2_F038
Finished  Ses01F_script03_2_F039
Finished  Ses01F_script03_2_F040
Finished  Ses01F_script03_2_M001
Finished  Ses01F_script03_2_M002
Finished  Ses01F_script03_2_M004
Finished  Ses01F_script03_2_M007
Finished  Ses01F_script03_2_M010
Finished  Ses01F_script03_2_M011
Finished  Ses01F_script03_2_M014
Finished  Ses01F_script03_2_M016
Finished  Ses01F_script03_2_M018
Finished  Ses01F_script03_2_M019
Finished  Ses01F_script03_2_M020
Finished  Ses01F_script03_2_M021
Finished  Ses01F_script03_2_M022
Finished  Ses01F_script03_2_M023
Finished  Ses01F_script03_2_M024
Finished  Ses01F_script03_2_M025
Finished  Ses01F_script03_2_M026
Finished  Ses01F_script03_2_M029
Finished  Ses01F_script03_2_M030
Finished  Ses01F_script03_2_M031
Finished  Ses01F_script03_2_M032
Finished  Ses01F_script03_2_M033
Finished  Ses01F_script03_2_M034
Finished  Ses01F_script03_2_M035
Finished  Ses01F_script03_2_M036
Finished  Ses01F_script03_2_M037
Finished  

Finished  Ses01M_script01_3_M043
Finished  Ses01M_script02_1_F005
Finished  Ses01M_script02_1_F009
Finished  Ses01M_script02_1_F014
Finished  Ses01M_script02_1_F017
Finished  Ses01M_script02_1_F022
Finished  Ses01M_script02_1_F023
Finished  Ses01M_script02_1_M007
Finished  Ses01M_script02_1_M009
Finished  Ses01M_script02_1_M010
Finished  Ses01M_script02_1_M011
Finished  Ses01M_script02_1_M018
Finished  Ses01M_script02_1_M023
Finished  Ses01M_script02_1_M028
Finished  Ses01M_script02_1_M030
Finished  Ses01M_script02_1_M037
Finished  Ses01M_script02_1_M039
Finished  Ses01M_script02_2_F000
Finished  Ses01M_script02_2_F010
Finished  Ses01M_script02_2_F011
Finished  Ses01M_script02_2_F013
Finished  Ses01M_script02_2_F014
Finished  Ses01M_script02_2_F015
Finished  Ses01M_script02_2_F016
Finished  Ses01M_script02_2_F018
Finished  Ses01M_script02_2_F019
Finished  Ses01M_script02_2_F022
Finished  Ses01M_script02_2_F024
Finished  Ses01M_script02_2_F030
Finished  Ses01M_script02_2_F031
Finished  

Finished  Ses02F_script01_3_M032
Finished  Ses02F_script01_3_M033
Finished  Ses02F_script01_3_M034
Finished  Ses02F_script01_3_M036
Finished  Ses02F_script02_1_F001
Finished  Ses02F_script02_1_F019
Finished  Ses02F_script02_1_F021
Finished  Ses02F_script02_1_M009
Finished  Ses02F_script02_1_M010
Finished  Ses02F_script02_1_M012
Finished  Ses02F_script02_1_M013
Finished  Ses02F_script02_1_M017
Finished  Ses02F_script02_1_M020
Finished  Ses02F_script02_1_M021
Finished  Ses02F_script02_1_M031
Finished  Ses02F_script02_1_M033
Finished  Ses02F_script02_1_M043
Finished  Ses02F_script02_1_M044
Finished  Ses02F_script02_2_F000
Finished  Ses02F_script02_2_F005
Finished  Ses02F_script02_2_F006
Finished  Ses02F_script02_2_F011
Finished  Ses02F_script02_2_F014
Finished  Ses02F_script02_2_F015
Finished  Ses02F_script02_2_F016
Finished  Ses02F_script02_2_F017
Finished  Ses02F_script02_2_F018
Finished  Ses02F_script02_2_F019
Finished  Ses02F_script02_2_F020
Finished  Ses02F_script02_2_F028
Finished  

Finished  Ses02M_impro08_F019
Finished  Ses02M_impro08_F025
Finished  Ses02M_impro08_F026
Finished  Ses02M_impro08_F027
Finished  Ses02M_impro08_F028
Finished  Ses02M_impro08_F030
Finished  Ses02M_impro08_M000
Finished  Ses02M_impro08_M002
Finished  Ses02M_impro08_M003
Finished  Ses02M_impro08_M004
Finished  Ses02M_impro08_M005
Finished  Ses02M_impro08_M007
Finished  Ses02M_impro08_M008
Finished  Ses02M_impro08_M009
Finished  Ses02M_impro08_M010
Finished  Ses02M_impro08_M011
Finished  Ses02M_impro08_M012
Finished  Ses02M_impro08_M013
Finished  Ses02M_impro08_M014
Finished  Ses02M_impro08_M015
Finished  Ses02M_impro08_M016
Finished  Ses02M_impro08_M017
Finished  Ses02M_impro08_M019
Finished  Ses02M_impro08_M020
Finished  Ses02M_impro08_M021
Finished  Ses02M_impro08_M022
Finished  Ses02M_impro08_M023
Finished  Ses02M_impro08_M024
Finished  Ses02M_impro08_M025
Finished  Ses02M_impro08_M026
Finished  Ses02M_impro08_M027
Finished  Ses02M_impro08_M028
Finished  Ses02M_impro08_M029
Finished  

...

Finished  Ses04F_script01_2_F010
Finished  Ses04F_script01_2_F013
Finished  Ses04F_script01_2_F015
Finished  Ses04F_script01_2_F016
Finished  Ses04F_script01_2_M000
Finished  Ses04F_script01_2_M001
Finished  Ses04F_script01_2_M004
Finished  Ses04F_script01_2_M007
Finished  Ses04F_script01_2_M008
Finished  Ses04F_script01_2_M017
Finished  Ses04F_script01_3_F009
Finished  Ses04F_script01_3_F010
Finished  Ses04F_script01_3_F012
Finished  Ses04F_script01_3_F013
Finished  Ses04F_script01_3_F014
Finished  Ses04F_script01_3_F015
Finished  Ses04F_script01_3_F023
Finished  Ses04F_script01_3_F024
Finished  Ses04F_script01_3_F030
Finished  Ses04F_script01_3_M008
Finished  Ses04F_script01_3_M009
Finished  Ses04F_script01_3_M010
Finished  Ses04F_script01_3_M011
Finished  Ses04F_script01_3_M014
Finished  Ses04F_script01_3_M018
Finished  Ses04F_script01_3_M019
Finished  Ses04F_script01_3_M020
Finished  Ses04F_script01_3_M023
Finished  Ses04F_script01_3_M024
Finished  Ses04F_script01_3_M026
Finished  

Finished  Ses04M_impro06_F019
Finished  Ses04M_impro06_M001
Finished  Ses04M_impro06_M002
Finished  Ses04M_impro06_M005
Finished  Ses04M_impro06_M006
Finished  Ses04M_impro06_M007
Finished  Ses04M_impro06_M008
Finished  Ses04M_impro06_M010
Finished  Ses04M_impro06_M011
Finished  Ses04M_impro06_M012
Finished  Ses04M_impro06_M013
Finished  Ses04M_impro06_M014
Finished  Ses04M_impro06_M015
Finished  Ses04M_impro06_M016
Finished  Ses04M_impro06_M018
Finished  Ses04M_impro06_M019
Finished  Ses04M_impro06_M020
Finished  Ses04M_impro07_F011
Finished  Ses04M_impro07_F023
Finished  Ses04M_impro07_F026
Finished  Ses04M_impro07_F036
Finished  Ses04M_impro07_M005
Finished  Ses04M_impro07_M007
Finished  Ses04M_impro07_M009
Finished  Ses04M_impro07_M010
Finished  Ses04M_impro07_M011
Finished  Ses04M_impro07_M012
Finished  Ses04M_impro07_M013
Finished  Ses04M_impro07_M025
Finished  Ses04M_impro07_M026
Finished  Ses04M_impro07_M029
Finished  Ses04M_impro07_M030
Finished  Ses04M_impro07_M032
Finished  

Finished  Ses05F_impro05_M017
Finished  Ses05F_impro05_M018
Finished  Ses05F_impro05_M022
Finished  Ses05F_impro05_M025
Finished  Ses05F_impro05_M028
Finished  Ses05F_impro05_M029
Finished  Ses05F_impro05_M030
Finished  Ses05F_impro05_M031
Finished  Ses05F_impro05_M032
Finished  Ses05F_impro05_M033
Finished  Ses05F_impro05_M034
Finished  Ses05F_impro05_M036
Finished  Ses05F_impro05_M037
Finished  Ses05F_impro05_M039
Finished  Ses05F_impro05_M040
Finished  Ses05F_impro05_M041
Finished  Ses05F_impro05_M044
Finished  Ses05F_impro05_M049
Finished  Ses05F_impro05_M050
Finished  Ses05F_impro05_M051
Finished  Ses05F_impro05_M052
Finished  Ses05F_impro05_M053
Finished  Ses05F_impro06_F000
Finished  Ses05F_impro06_F001
Finished  Ses05F_impro06_F002
Finished  Ses05F_impro06_F003
Finished  Ses05F_impro06_F005
Finished  Ses05F_impro06_F006
Finished  Ses05F_impro06_F007
Finished  Ses05F_impro06_F008
Finished  Ses05F_impro06_F009
Finished  Ses05F_impro06_F010
Finished  Ses05F_impro06_F011
Finished  

Finished  Ses05F_script03_2_M019
Finished  Ses05F_script03_2_M020
Finished  Ses05F_script03_2_M021
Finished  Ses05F_script03_2_M022
Finished  Ses05F_script03_2_M024
Finished  Ses05F_script03_2_M025
Finished  Ses05F_script03_2_M026
Finished  Ses05F_script03_2_M037
Finished  Ses05F_script03_2_M038
Finished  Ses05F_script03_2_M039
Finished  Ses05F_script03_2_M040
Finished  Ses05F_script03_2_M041
Finished  Ses05F_script03_2_M042
Finished  Ses05F_script03_2_M043
Finished  Ses05M_impro01_F000
Finished  Ses05M_impro01_F001
Finished  Ses05M_impro01_F002
Finished  Ses05M_impro01_F003
Finished  Ses05M_impro01_F004
Finished  Ses05M_impro01_F006
Finished  Ses05M_impro01_F009
Finished  Ses05M_impro01_F012
Finished  Ses05M_impro01_F021
Finished  Ses05M_impro01_F023
Finished  Ses05M_impro01_M005
Finished  Ses05M_impro01_M010
Finished  Ses05M_impro01_M015
Finished  Ses05M_impro02_F012
Finished  Ses05M_impro02_F013
Finished  Ses05M_impro02_F020
Finished  Ses05M_impro02_F021
Finished  Ses05M_impro02_M00

Finished  Ses05M_script03_2_M029
Finished  Ses05M_script03_2_M030
Finished  Ses05M_script03_2_M031
Finished  Ses05M_script03_2_M032
Finished  Ses05M_script03_2_M033
Finished  Ses05M_script03_2_M034
Finished  Ses05M_script03_2_M035
Finished  Ses05M_script03_2_M036
Finished  Ses05M_script03_2_M037
Finished  Ses05M_script03_2_M038
Finished  Ses05M_script03_2_M039
Finished  Ses05M_script03_2_M040
Finished  Ses05M_script03_2_M041
Finished  Ses05M_script03_2_M042
Finished  Ses05M_script03_2_M043
Finished  Ses05M_script03_2_M044
Finished  Ses05M_script03_2_M045
All done


In [276]:
# scalfull_feature_table
# scalwc_feature_table
# scalcw_feature_table

In [240]:
try:
    print('Saving Pickle')
    with open('pickles/patterns/scaled_pattern_features4emo.pickle','wb') as f:
        save = {
            'full_feature_table' : scalfull_feature_table,
            'wc_feature_table' : scalwc_feature_table,
            'cw_feature_table' : scalcw_feature_table
        }
        pickle.dump(save,f,pickle.HIGHEST_PROTOCOL)
        print('Successfully saved in scaled_pattern_features4emo.pickle')
except Exception as e:
    print('Unable to save data to pickle', e)
    

Saving Pickle
Successfully saved in scaled_pattern_features4emo.pickle


In [ ]:

    
#     if(pattern_name not in feature_table):
#         feature_table[pattern_name] = {}

In [348]:
'pickles/patterns/cw_matrix.pickle'

'pickles/patterns/cw_matrix.pickle'